In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
from string import digits
import nltk; nltk.download('stopwords')
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
hotel_review = pd.read_csv('Hotel_Reviews.csv')
hotel_review.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397.0,1403.0,Only the park outside of the hotel was beauti...,11.0,7.0,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0.0,1403.0,No real complaints the hotel was great great ...,105.0,7.0,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42.0,1403.0,Location was good and staff were ok It is cut...,21.0,9.0,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210.0,1403.0,Great location in nice surroundings the bar a...,26.0,1.0,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140.0,1403.0,Amazing location and building Romantic setting,8.0,3.0,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
# 나라 중복값 확인
w_count  = {}
for lst in hotel_review['Reviewer_Nationality']:
    try: w_count[lst] += 1
    except: w_count[lst]=1
print(w_count)

{' Russia ': 150, ' Ireland ': 1097, ' Australia ': 1724, ' United Kingdom ': 23093, ' New Zealand ': 243, ' Poland ': 133, ' Belgium ': 387, ' Norway ': 179, ' France ': 510, ' Italy ': 324, ' Canada ': 596, ' Netherlands ': 510, ' Germany ': 450, ' United States of America ': 2670, ' Hungary ': 68, ' Israel ': 344, ' Switzerland ': 554, ' India ': 220, ' United Arab Emirates ': 688, ' Malaysia ': 174, ' Denmark ': 122, ' South Africa ': 275, ' Rwanda ': 1, ' Finland ': 99, ' Portugal ': 115, ' Luxembourg ': 80, ' Spain ': 260, ' Malta ': 77, ' China ': 259, ' Turkey ': 255, ' Ukraine ': 44, ' Gibraltar ': 26, ' Argentina ': 25, ' Abkhazia Georgia ': 14, ' Isle of Man ': 37, ' Sweden ': 229, ' Croatia ': 51, ' Saudi Arabia ': 555, ' Lithuania ': 29, ' Slovenia ': 24, ' Panama ': 12, ' Austria ': 111, ' Brazil ': 104, ' Taiwan ': 64, ' Iceland ': 59, ' Iran ': 40, ' Liechtenstein ': 1, ' Chile ': 21, ' Hong Kong ': 280, ' United States Minor Outlying Islands ': 10, ' Morocco ': 11, ' O

In [4]:
# 대륙별 행 추출_아시아
hotel_review_Asia = hotel_review[['Reviewer_Nationality',"Negative_Review","Positive_Review"]]
Id = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " India "]
Ma = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Malaysia "]
Ch = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " China "]
Tu = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Turkey "]
Au = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Australia "]
Nz = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " New Zealand "]
Th = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Thailand "]
Si = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Singapore "]
Ja = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Japan "]
Sk = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " South Korea "]
Ta = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Taiwan "]
Hk = hotel_review_Asia[hotel_review_Asia['Reviewer_Nationality']  == " Hong Kong "]
ASIA = pd.concat([Id,Ma,Ch,Tu,Au,Nz,Th,Si,Ja,Sk,Ta,Hk])
ASIA

,Reviewer_Nationality,Negative_Review,Positive_Review
45,India,Bathroom lighting could have been brighter Th...,The location of the hotel is super opening ou...
247,India,The beds were a little small and very low,No Positive
429,India,No Negative,Room as advertised excellent location a 2 min...
733,India,No Negative,Help from staff in a medical emergency
1907,India,inhouse restaurant isnt that great,location good rooms
2148,India,The carpeting in the corridor out side the ro...,The Handy Phone was a nice touch Helped a lot...
2216,India,The room is way too small No matter how well ...,Great location for the property Close to Hyde...
2313,India,The staff was very in co operative and the ro...,The only good thing was the location
2329,India,Everything else you don t expect big and lavi...,Location is good walking distance from Paddin...
2375,India,After hauling 4 suitcases across London we we...,Location was really close to train station an...


In [0]:
Negative = ASIA.iloc[:,1]
Positive = ASIA.iloc[:,2]

In [6]:
Negative = [re.sub("\'", "", str(sent)) for sent in Negative]
Negative = [re.sub("No Negative'", "", str(sent)) for sent in Negative]
pprint(Negative[:1])

[' Bathroom lighting could have been brighter The airconditioning of our room '
 '303 could not be turned down lower than 23 degrees celcius The refrigerator '
 'in our room was not cooling there was no thermostat The toiletries were not '
 'replenished shampoo shower gel dispenser was empty and was not replenished '
 'on the second day either ']


In [7]:
Positive = [re.sub("\'", "", str(sent1)) for sent1 in Positive]
Positive = [re.sub("No Positive'", "", str(sent1)) for sent1 in Positive]
pprint(Positive[:1])

[' The location of the hotel is super opening out to Oosterpark The staff was '
 'very helpful and friendly The bed was very comfortable The room was spacious '
 'enough for 2 people ']


In [0]:
import gensim 
from gensim.models import Word2Vec

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words1 = list(sent_to_words(Negative))
data_words2 = list(sent_to_words(Positive))

print(data_words1[:1])
print(data_words2[:1])

[['bathroom', 'lighting', 'could', 'have', 'been', 'brighter', 'the', 'airconditioning', 'of', 'our', 'room', 'could', 'not', 'be', 'turned', 'down', 'lower', 'than', 'degrees', 'celcius', 'the', 'refrigerator', 'in', 'our', 'room', 'was', 'not', 'cooling', 'there', 'was', 'no', 'thermostat', 'the', 'toiletries', 'were', 'not', 'replenished', 'shampoo', 'shower', 'gel', 'dispenser', 'was', 'empty', 'and', 'was', 'not', 'replenished', 'on', 'the', 'second', 'day', 'either']]
[['the', 'location', 'of', 'the', 'hotel', 'is', 'super', 'opening', 'out', 'to', 'oosterpark', 'the', 'staff', 'was', 'very', 'helpful', 'and', 'friendly', 'the', 'bed', 'was', 'very', 'comfortable', 'the', 'room', 'was', 'spacious', 'enough', 'for', 'people']]


In [10]:
%%time
model1 = Word2Vec(data_words1)

CPU times: user 530 ms, sys: 7.06 ms, total: 537 ms
Wall time: 333 ms


In [11]:
%%time
model2 = Word2Vec(data_words2)

CPU times: user 507 ms, sys: 6.29 ms, total: 513 ms
Wall time: 313 ms


In [31]:
model1.wv.similarity('service', "room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9995044

In [32]:
model1.wv.similarity('location', "room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9995245

In [33]:
model1.wv.similarity('service', "travel")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.99949396

In [35]:
# 부정적 리뷰 토픽 연관단어 도출
model1.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('off', 0.9999016523361206),
 ('also', 0.9998975992202759),
 ('with', 0.9998959302902222),
 ('which', 0.999893307685852),
 ('people', 0.9998897314071655),
 ('and', 0.999885082244873),
 ('for', 0.9998837113380432),
 ('from', 0.9998797178268433),
 ('breakfast', 0.9998788237571716),
 ('than', 0.9998758435249329)]

In [36]:
model1.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bed', 0.9997918009757996),
 ('too', 0.9997463226318359),
 ('bathroom', 0.9997434616088867),
 ('very', 0.9997321367263794),
 ('was', 0.9997041821479797),
 ('little', 0.9996824860572815),
 ('bit', 0.999681830406189),
 ('shower', 0.9996591806411743),
 ('rooms', 0.9996573328971863),
 ('tiny', 0.9996545910835266)]

In [37]:
model1.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('quite', 0.999886155128479),
 ('area', 0.9998762011528015),
 ('people', 0.9998703002929688),
 ('just', 0.9998691082000732),
 ('noisy', 0.9998619556427002),
 ('really', 0.9998580813407898),
 ('without', 0.9998519420623779),
 ('service', 0.999850869178772),
 ('old', 0.9998503923416138),
 ('price', 0.9998491406440735)]

In [38]:
model1.wv.most_similar("transport")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('access', 0.9994739294052124),
 ('open', 0.9994422793388367),
 ('didn', 0.9994398355484009),
 ('after', 0.9994338154792786),
 ('by', 0.9994262456893921),
 ('being', 0.9994248747825623),
 ('back', 0.999412477016449),
 ('into', 0.9994069933891296),
 ('car', 0.9994068145751953),
 ('more', 0.9994059801101685)]

In [39]:
# 긍정적 리뷰 토픽 연관단어 도출
model2.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('facilities', 0.9997658729553223),
 ('amazing', 0.9997528195381165),
 ('modern', 0.9997255802154541),
 ('welcoming', 0.9997169971466064),
 ('are', 0.999705970287323),
 ('quiet', 0.9996815919876099),
 ('really', 0.9996749758720398),
 ('lovely', 0.9996562600135803),
 ('rooms', 0.9996433258056641),
 ('beds', 0.9996392726898193)]

In [40]:
model2.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('spacious', 0.999547004699707),
 ('bathroom', 0.9995203018188477),
 ('rooms', 0.9994654655456543),
 ('beds', 0.9993888735771179),
 ('extremely', 0.9992818832397461),
 ('super', 0.9992531538009644),
 ('breakfast', 0.9992529153823853),
 ('size', 0.9992381930351257),
 ('modern', 0.9992203712463379),
 ('bed', 0.9991369843482971)]

In [41]:
model2.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('good', 0.9984912872314453),
 ('great', 0.9983866214752197),
 ('and', 0.9983680844306946),
 ('excellent', 0.997833788394928),
 ('nice', 0.9968424439430237),
 ('the', 0.9962975382804871),
 ('comfortable', 0.9962686896324158),
 ('breakfast', 0.9962402582168579),
 ('is', 0.9961481094360352),
 ('staffs', 0.996056318283081)]

In [42]:
model2.wv.most_similar("transport")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('kensington', 0.9997055530548096),
 ('lots', 0.9996972680091858),
 ('nearby', 0.9996722340583801),
 ('many', 0.9996660351753235),
 ('proximity', 0.9996626377105713),
 ('right', 0.9996550679206848),
 ('shops', 0.9996341466903687),
 ('subway', 0.9996126890182495),
 ('cafes', 0.9996094107627869),
 ('queensway', 0.9995921850204468)]